In [30]:
from fractions import Fraction,  gcd


def format_state_probabilities(terminal_states,  terminal_state_probabilities):
    """ Function changes the normalized state probabilities into format numerators-denominator """

    # Remove states that are not terminal.
    filtered_state_probabilities = []
    for i in range(len(terminal_states)):
        if (terminal_states[i]):
           filtered_state_probabilities.append(terminal_state_probabilities[i])

    # Special case  0: no states!
    if (len(filtered_state_probabilities) == 0):
        print("There are no final states!")
        return [1]

    numerators = []
    denominators = []
    fractions = [ Fraction(prob).limit_denominator() for prob in filtered_state_probabilities]
    #print("Fractions: {}".format(fractions))
    
    # Handle separatelly numerators and denominators.
    for frac in fractions:
        numerators.append(frac.numerator)
        denominators.append(frac.denominator)
    print("numerators: {}".format(numerators))
    print("denominators: {}".format(denominators))

    # Calculate factors
    max_den = max(denominators)
    factors = [max_den // den for den in denominators]
    print("factors: {}".format(factors))
    
    # Bring to common denominator.
    final_numerators = [num * fac for num, fac in zip(numerators, factors)]
    print("final_numerators: {}".format(final_numerators))
    
    # Sanity check
    if (sum(final_numerators) != denominators[0] ):
        print("Error! Numerators do not sum to denominator!")

    # Format output
    output = []
    
    output = [int(el) for el in final_numerators]
    output.append(max_den)
    return output
  


# Maximum number of considered transitions.
MAX_STEPS = 10000
    
def answer(m):
    """ Calculates the probabilities of reaching the terminal states"""

    # Edge case 0: empty matrix.
    if (len(m) == 0):
        print("Input matrix is empty")
        return []
        
    # Edge case 2: 1d matrix.
    if (len(m) == 1):
        print("Input matrix is 1d")
        return [1, 1]

    # Edge case 2: badly formed matrix?
    no_states = len(m)
    #for i in range(no_states):
    #    if (len(m[i]) != no_states):
    #        print("Input matrix is not square (length of row {} is {} != {})".format(i,  len(m[i]),  no_states))
    #        return []
    
    # Calculate tmp variable - sums of rows
    row_sums = [sum(i)  for i in m]
    #print("row_sums=", row_sums)
 
    # Get terminal states
    terminal_states = []
    for i in range(no_states):
        # If there are no outputs.
        if (row_sums[i] == 0):
            terminal_states.append(True)
        # Or all outputs lead to the same node (diagonal):
        elif (len(m[i]) >i) and (row_sums[i] == m[i][i]) :
            terminal_states.append(True)
        else:
            terminal_states.append(False)
    print("terminal states=",terminal_states)

    # Edge case 3: no terminal states
    if (len([ state for state in terminal_states if state == True ]) == 0):
        print("There are no terminal states")
        return []

    # Form the Markov transition matrix.
    transition_matrix = []
    for i in range(no_states):
        if (row_sums[i] != 0):
            transition_matrix.append( [j/row_sums[i] for j in m[i]])
        else:
            # All zeros! In that case let's assume that the ore will stay in that state
            # Add self-transitions for terminal nodes: i=>i  with prob = 1
            transition_matrix.append(m[i])
            transition_matrix[i][i] =1
    print("transition_matrix=", transition_matrix)

    # Initial state.
    prev_ore_state = transition_matrix[0]
    print("initial_ore_state=", prev_ore_state)

    # Lets "evolve" the ore, starting from the first state, using PageRank.
    for i in range(MAX_STEPS):
        ore_state = []
        # Get columns from 
        for col in zip(*transition_matrix):
            # Sum all probabilities leading to a given state
            ore_state.append(sum([a*b for a,b in zip(prev_ore_state,  col)]))
            #print(zip(prev_state) * col)
        prev_ore_state = ore_state
    
    print("final_ore_state=", ore_state)
    
    # Return formated probabilities
    return format_state_probabilities(terminal_states,  ore_state)


if __name__ == "__main__":
    ore_trans_mat = [
      [0,1,0,0,0,1],  # s0, the initial state, goes to s1 and s5 with equal probability
      [4,0,0,3,2,0],  # s1 can become s0, s3, or s4, but with different probabilities
      [0,0,0,0,0,0],  # s2 is terminal, and unreachable (never observed in practice)
      [0,0,0,0,0,0],  # s3 is terminalnumerators
      [0,0,0,0,0,0],  # s4 is terminal 
      [0,0,0,0,0,0],  # s5 is terminal
    ]
    
    ore_trans_mat = [
        list(range(0,10)),
        list(range(1,11)),
        list(range(2,12)),
        list(range(3,13)),
        list(range(4,14)),
        [0, 0, 0, 0, 0, 1000, 0, 0, 0, 0],
        list(range(6,16)),
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        list(range(8,18)),
        list(range(9,19))
    ]
    
    #ore_trans_mat = [
    #    [1, 0, 0, 0],
    #    [0, 1, 0, 0],
    #    [0, 0, 1, 0],
    #    [0, 0, 0, 1]
    #]
    
    #ore_trans_mat = [
    #    [1000, 2000, 3000, 4000],
    #    [0, 1000, 0, 0],
    #    [0, 0, 10001, 0],
    #    [0, 0, 0, 16000]
    #]
    
    
    #ore_trans_mat =   [[0, 2, 1, 0, 0], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

    #ore_trans_mat = [[0, 1, 0, 0, 0, 1], [4, 0, 0, 3, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
    
    # Tricky cases!
    #ore_trans_mat =   [[], []]

    #ore_trans_mat =   [[0, 1,  0], [0, 1,  0],  [0, 1, 0]]

    #ore_trans_mat =   [[0,  2,  3,  4]]
    
    #ore_trans_mat =   [[0, 2], [1],  [0], [0, 0]]
    
    #ore_trans_mat =   [[1]]
    #ore_trans_mat =   [[0,  0],  [0, 1]]
    #ore_trans_mat = [[0,1,0,1], [1, 0, 0, 1], [0, 0, 0, 0], [0, 1, 1, 0]]
    
    #ore_trans_mat

    print("ore_trans_mat=",ore_trans_mat)

    print("answer =",answer(ore_trans_mat))
    



ore_trans_mat= [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [0, 0, 0, 0, 0, 1000, 0, 0, 0, 0], [6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8, 9, 10, 11, 12, 13, 14, 15, 16, 17], [9, 10, 11, 12, 13, 14, 15, 16, 17, 18]]
terminal states= [False, False, False, False, False, True, False, True, False, False]
transition_matrix= [[0.0, 0.022222222222222223, 0.044444444444444446, 0.06666666666666667, 0.08888888888888889, 0.1111111111111111, 0.13333333333333333, 0.15555555555555556, 0.17777777777777778, 0.2], [0.01818181818181818, 0.03636363636363636, 0.05454545454545454, 0.07272727272727272, 0.09090909090909091, 0.10909090909090909, 0.12727272727272726, 0.14545454545454545, 0.16363636363636364, 0.18181818181818182], [0.03076923076923077, 0.046153846153846156, 0.06153846153846154, 0.07692307692307693, 0.09230769230769231, 0.1076923076923077, 0.